In [11]:
# 1. IMPORTS
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score
from joblib import dump

# 2. LOAD DATA
df = pd.read_csv('player_data.csv')

# 3. CREATE INJURY RISK CATEGORY (TARGET VARIABLE)
def categorize_risk(total):
    if total <= 1:
        return "Low"
    elif total <= 3:
        return "Low-Medium"
    elif total <= 5:
        return "Medium"
    elif total <= 7:
        return "Medium-High"
    else:
        return "High"

df["Injury_Risk_Level"] = df["Total_Injuries"].apply(categorize_risk)

# 4. ENCODE TARGET LABEL
le = LabelEncoder()
df["Injury_Risk_Label"] = le.fit_transform(df["Injury_Risk_Level"])

# 5. FEATURES & TARGET
features = [
    'Age', 'Height_in_Inches', 'Weight_in_Pounds', 'Games_Played',
    'Minutes_Played', 'Field_Goals_Attempted', 'Three_Point_Field_Goals_Attempted',
    'Free_Throws_Attempted', 'Steals', 'Blocks', 'Fouls'
]
X = df[features]
y = df["Injury_Risk_Label"]

# 6. SPLIT DATA
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 7. MODEL BENCHMARKING
models = {
    "RandomForest": RandomForestClassifier(random_state=42),
    "SVM": SVC(kernel='rbf', probability=True),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
}

from sklearn.utils.multiclass import unique_labels

for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    print(f"\n=== {name} ===")
    print("Accuracy:", accuracy_score(y_test, preds))

    # Get only labels that are present in y_test or preds
    labels_in_data = unique_labels(y_test, preds)

    # Print classification report safely with matching label names
    print(classification_report(
        y_test,
        preds,
        labels=labels_in_data,
        target_names=le.inverse_transform(labels_in_data),
        zero_division=0
    ))


# 8. TRAIN FINAL MODEL & SAVE
best_model = RandomForestClassifier(random_state=42)
best_model.fit(X, y)

dump(best_model, "rf_injury_model.joblib")
dump(le, "risk_label_encoder.joblib")

print("✅ Model and label encoder saved for Django.")



=== RandomForest ===
Accuracy: 0.7884615384615384
              precision    recall  f1-score   support

        High       0.00      0.00      0.00         1
         Low       0.91      0.91      0.91        44
  Low-Medium       0.33      0.17      0.22         6
      Medium       0.00      0.00      0.00         0
 Medium-High       0.00      0.00      0.00         1

    accuracy                           0.79        52
   macro avg       0.25      0.22      0.23        52
weighted avg       0.81      0.79      0.79        52


=== SVM ===
Accuracy: 0.7884615384615384
              precision    recall  f1-score   support

        High       0.00      0.00      0.00         1
         Low       0.89      0.91      0.90        44
  Low-Medium       0.14      0.17      0.15         6
 Medium-High       0.00      0.00      0.00         1

    accuracy                           0.79        52
   macro avg       0.26      0.27      0.26        52
weighted avg       0.77      0.79     

/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [11:40:30] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


✅ Model and label encoder saved for Django.


In [13]:
# FINAL MODEL SELECTION (choose your best model)
final_model = RandomForestClassifier(random_state=42)  # or XGBClassifier(...) if that performed better
final_model.fit(X, y)  # Train on full dataset

# SAVE MODEL & LABEL ENCODER
from joblib import dump

dump(final_model, 'rf_injury_model.joblib')
dump(le, 'risk_label_encoder.joblib')

print("✅ Saved model and label encoder to use in Django app.")


✅ Saved model and label encoder to use in Django app.
